In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!python -m spacy download pt
!pip install --upgrade tables

✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/pt_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/pt
You can now load the model via spacy.load('pt')
Requirement already up-to-date: tables in /usr/local/lib/python3.6/dist-packages (3.6.1)


In [ ]:
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords 
import re
import spacy
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import dateutil
import seaborn as sns
from collections import OrderedDict, Counter
import os
import networkx as nx
import matplotlib.pyplot as plt
from nltk import word_tokenize
import string
from datetime import datetime
import plotly.graph_objs as go

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df_tweet = pd.read_csv('/content/drive/My Drive/Projeto Ana/Utils/Análise 2020/tweets_total.csv', encoding = 'utf-8')
df_tweet = df_tweet.dropna()

df_news = pd.read_csv('/content/drive/My Drive/Projeto Ana/Entrega_v1/Análise_Mídia/mediaframe_export.csv', encoding = 'utf-8')   
df_news = df_news.dropna()

In [ ]:
def word_transform(words):
    words = words.split()
    total = [x.lower() for x in words]
    manual_list_stop_words = ['só','disse','ser','sobre','ainda','se','além', 'outros','porque','assim','ter', 'access_time', 'more_horiz',
                             'out','jun','publicado','diz','exame','patrocinado','ago','vip','abr','http','vai','HTTP'
                             'jul','jan','abr','set','nov','dez','conteúdo','mar','maio','estadão','marketing','pictwittercom','dia','mil','pra',
                             'fev','paulo', 'the','após', 'redação', 'foto', 'feira', 'segundo', 'nesta', 'sexta','contra','governo','dilma'
            'onde','pode','sábado','segunda','três', 'fazer','todos','parte','local', 'região', 'disse','quarta','nova','novo',
            'via','veja','neste','nm','aponta','mostra','rt','vejaabrilcombr','folha','vai','desde','do','multimidia','vejacom','blog', '-', '@','"','...','dilma']
    stop_words = stopwords.words('portuguese')
    for i in manual_list_stop_words:
        stop_words.append(i)
    
    total = [x for x in total if x not in stop_words]
    total = [re.sub(r'[,\.!?()]', '', x) for x in total]
    total = [x for x in total if not re.search(r"\d", x)]
    
    long_string = (" ").join(total)
    return long_string

In [ ]:
df_news['text'] = df_news['text'].apply(word_transform)

df_tweet['tweet'] = df_tweet['tweet'].apply(word_transform)

In [ ]:
df_news_work = df_news[['publication_date','text']]
df_news_work.columns = ['Data','Texto']

df_tweet_work = df_tweet[['date','tweet']]
df_tweet_work.columns = ['Data','Tweet']

In [ ]:
df_tweet_work['Data'] = df_tweet_work['Data'].apply(lambda x: datetime.strptime(str(x), '%Y-%m-%d'))
df_tweet_work['Data'] = df_tweet_work['Data'].apply(lambda x: x.to_period('M'))
df_tweet_work['Data'] = df_tweet_work['Data'].apply(lambda x: x.to_timestamp())

In [ ]:
def format_data(x):
  datetime_object = dateutil.parser.parse(x)

  # formatar o datetime para o formato desejado (dia/mês/ano)
  dataFormatada = datetime_object.strftime("%d/%m/%Y")
  return dataFormatada

df_news_work['Data'] = df_news_work['Data'].apply(format_data)
df_news_work['Data'] = df_news_work['Data'].apply(lambda x: datetime.strptime(str(x), '%d/%m/%Y'))
df_news_work['Data'] = df_news_work['Data'].apply(lambda x: x.to_period('M'))
df_news_work['Data'] = df_news_work['Data'].apply(lambda x: x.to_timestamp())

In [ ]:
df_news_work = df_news_work.groupby(['Data'])['Texto'].apply(' '.join).reset_index()

df_tweet_work = df_tweet_work.groupby(['Data'])['Tweet'].apply(' '.join).reset_index()

In [ ]:
df_total = pd.merge(df_news_work, df_tweet_work, on = ['Data'])

In [ ]:
#df_total.to_hdf('/content/drive/My Drive/Projeto Ana/Utils/Análise 2020/total_midia_tweet.hdf', key = 'hdf')
df_total=pd.read_hdf('/content/drive/My Drive/Projeto Ana/Utils/Análise 2020/total_midia_tweet.hdf', key = 'hdf')

In [ ]:
nlp = spacy.load('pt')
nlp.max_length = 2800000

In [ ]:
df_total = df_total.iloc[20:]
df_total = df_total.reset_index(drop = True)

In [ ]:
def sim(doc1, doc2):
  return doc1.similarity(doc2)

In [ ]:
array_2 = []
nlp_doc1 = None
nlp_doc2 = None
for i in tqdm(range(28)):
  doc1 = df_total['Tweet'].iloc[0:1].values[0]
  doc1 = doc1.split()[:100000]
  doc1 = ' '.join(doc1)

  doc2 = df_total['Texto'].iloc[0:1].values[0]
  doc2 = doc2.split()[:100000]
  doc2 = ' '.join(doc2)

  data = df_total['Data'].iloc[0:1].values[0]

  doc1 = nlp(doc1)
  doc2 = nlp(doc2)

  df_total = df_total.drop(df_total.index[[0]])
  df_total = df_total.reset_index(drop = True)
  sim = doc1.similarity(doc2)

  del doc1
  del doc2
  


 
  array_2.append([data,sim])

/usr/lib/python3.6/runpy.py:193: ModelsWarning:

[W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.

/usr/lib/python3.6/runpy.py:193: ModelsWarning:

[W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.

/usr/lib

/usr/lib/python3.6/runpy.py:193: ModelsWarning:

[W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.



In [ ]:
pd.DataFrame(array_2).to_csv(c)

In [ ]:
doc1 = df_total['Tweet'].iloc[0:1].values[0]
doc1 = doc1.split()[:500000]
doc1 = ' '.join(doc1)

doc2 = df_total['Texto'].iloc[0:1].values[0]
doc2 = doc2.split()[:500000]
doc2 = ' '.join(doc2)

In [ ]:
 data = df_total['Data'].iloc[0:1].values[0]

nlp_doc1 = nlp(doc1)
nlp_doc2 = nlp(doc2)

df_total = df_total.drop(df_total.index[[0]])
df_total.reset_index(drop = True)

,Data,Texto,Tweet
0,2013-03-01,retomada crescimento econômico somada maior us...,» azenha: globo parecem estar perto vitória @i...
1,2013-04-01,"""essa brecha criação partidos utilizada partid...",alerta brasil: nazareno fonteles -o petista qu...
2,2013-05-01,cimi levou conta outras ações registradas orga...,@roxmo ano propôs acabar farra arrendamentos b...
3,2013-06-01,deve voltar avaliar possibilidade pronunciamen...,grandes decepções relação dilma retrocedemos r...
4,2013-07-01,ex-governador josé serra psdb afirmou tarde de...,escândalo tucano pipocando galera demonizando ...
5,2013-08-01,falcatruas liquidação banco rural rural nome l...,andar carruagem faltara cela especial ex-integ...
6,2013-09-01,brasília banco central bc pessimista crescimen...,“a espionagem nsa grave resposta ineficaz” @di...
7,2013-10-01,globo tamanho texto brasília trânsito livre pa...,pessoal apoia começa coçar cabeça verem luz ve...
8,2013-11-01,processo interrompido eleição pt limitou mante...,explicado contrato ibope r$ milhões única estr...
9,2013-12-01,reportagem jornal nacional tv globo jose eugen...,@edvaledval: retrospectiva desaprovação saúde ...


In [ ]:
nlp_doc1.similarity(nlp_doc2)

/usr/lib/python3.6/runpy.py:193: ModelsWarning:

[W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.



0.9549698890325325

In [ ]:
df1 = pd.read_csv('/content/drive/My Drive/Projeto Ana/Utils/Análise 2020/2014-09_similaridade.csv')
df2 = pd.read_csv('/content/drive/My Drive/Projeto Ana/Utils/Análise 2020/2014-08_similaridade.csv')


In [ ]:
df_total = pd.concat([df2,df1])

In [ ]:
df_total

,Unnamed: 0,0,1
0,0,2014-09-01,0.954463
1,1,2014-10-01,0.973430
2,2,2014-11-01,0.967619
3,3,2014-12-01,0.966987
4,4,2015-01-01,0.969573
5,5,2015-02-01,0.963006
6,6,2015-03-01,0.955877
7,7,2015-04-01,0.964053
8,8,2015-05-01,0.973276
9,9,2015-06-01,0.965090


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_total['0'], y=df_total['1'], name = 'Similaridade Tweets-Notícias'))
fig.update_xaxes(rangeslider_visible=True)
fig.show()